In [0]:
from pyspark.sql import functions as F

In [0]:
dbutils.fs.mounts()

In [0]:
mount_path = "/mnt/rainbowstorage"
file_path = "raw_datasets/Employee.csv"

emp_df = spark.read.format("csv").option("header", True).load(f"{mount_path}/{file_path}")
emp_df.display()

In [0]:
def convert_col_case(df):
    columns = df.columns
    for each_col in columns:
        df = df.withColumnRenamed(each_col, each_col.lower())
    return df


In [0]:
dummy_df = convert_col_case(emp_df)
display(dummy_df)

In [0]:
def convert_col_case2(df):
    columns = [cols.lower() for cols in df.columns]
    return df.toDF(*columns)

In [0]:
new_emp_df = convert_col_case2(emp_df)
new_emp_df.display()

In [0]:
new_emp_df = emp_df.withColumn("Status", F.lit(True))

In [0]:
new_emp_df.write.format("delta").option("mergeSchema", True).mode("append").save(f"{mount_path}/delta_datasets/employee")

In [0]:
df = spark.read.format("delta").load(f"{mount_path}/delta_datasets/employee")
df.display()

In [0]:
display(df.filter(F.col("status").isNull()))

In [0]:
df = spark.read.format("delta").option("versionAsOf", 0).load(f"{mount_path}/delta_datasets/employee")
df.display()

In [0]:
display(df.filter(F.col("status").isNull()))

In [0]:
history_df = spark.sql(f"DESCRIBE HISTORY '{mount_path}/delta_datasets/employee'")
display(history_df)

In [0]:
%sql
describe history '/mnt/rainbowstorage/delta_datasets/employee'

In [0]:
new_emp_df = new_emp_df.filter(F.col("city") == "Bangalore")
new_emp_df.write.format("delta").option("mergeSchema", True).mode("overwrite").save(f"{mount_path}/delta_datasets/employee")

In [0]:
history_df = spark.sql(f"DESCRIBE HISTORY '{mount_path}/delta_datasets/employee'")
display(history_df)

In [0]:
df = spark.read.format("delta").load(f"{mount_path}/delta_datasets/employee")
df.display()

In [0]:
df = spark.read.format("delta").option("versionAsOf", 1).load(f"{mount_path}/delta_datasets/employee")
df.display()